# Launch Sites Locations Analysis with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

#### Import libraries

In [1]:
!pip install folium==0.7.0

In [2]:
!pip install js

In [3]:
import folium
import pandas as pd

#Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
#Import folium MousePosition plugin
from folium.plugins import MousePosition
#Import folium DivIcon plugin
from folium.features import DivIcon

### Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [4]:
# Download and read the `spacex_launch_geo.csv`

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup("NASA Johnson Space Center"))
#Create a blue circle at NASA Johnson Space Center's coordinates with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    #Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`.

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for i in range(len(launch_sites_df)):
    lat = launch_sites_df.loc[i, 'Lat']
    lon = launch_sites_df.loc[i, 'Long']
    launch_site = launch_sites_df.loc[i, 'Launch Site']
    print(lat, lon, launch_site)
    #Add circle
    new_circle = folium.Circle([lat, lon], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_site))
    # Add marker
    new_marker = folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>' % launch_site,
        )
    )
    site_map.add_child(new_circle)
    site_map.add_child(new_marker)

site_map

28.56230197 -80.57735648 CCAFS LC-40
28.56319718 -80.57682003 CCAFS SLC-40
28.57325457 -80.64689529 KSC LC-39A
34.63283416 -120.6107455 VAFB SLC-4E


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
    - Yes, they are. The distance to orbital space is shorter from the equator.

*   Are all launch sites in very close proximity to the coast?
    - Yes, they also are in close proximity to the coast. This is in the event of failures, there will be less casualties and they can reclaim the fallen debris from the ocean.
Also please try to explain your findings.

### Mark the Success/ Failed launches for each site ont he map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [10]:
#Create a marker cluster object
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [11]:
#Apply a function to check the value of class column
#If class=1, marker_color value will be green
#If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

In [12]:
#Add marker cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    coord = [record['Lat'], record['Long']]
    marker = folium.Marker(
        coord,
        icon=folium.Icon(
            color='white',
            icon_color=record['marker_color']),
            popup=record['Launch Site'])
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

**Observation**: We can see that site KSC LC-39A has a higher success rate.


### Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
#Find coordinate of the closest coastline / railway point
launch_site = [28.563197, -80.576820]
coastline_coord = [28.56334, -80.56799]
#railway_coord = [28.57157, -80.58533]
distance_coastline = calculate_distance(launch_site[0], launch_site[1], coastline_coord[0], coastline_coord[1])
print(distance_coastline)

0.8627671182499878


In [16]:
# Create and add a folium.Marker on the selected closest coastline point on the map
# Display the distance between the coastline point and launch site using the icon property
# for example
distance_marker = folium.Marker(
    coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color=#d35000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [launch_site,coastline_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [18]:
railway_coord = [28.57157, -80.58533]
highway_coord = [28.56129, -80.57036]
city_coord = [28.61248, -80.80826]
distance_railway = calculate_distance(launch_site[0], launch_site[1], railway_coord[0], railway_coord[1])
print(f"Railway: {distance_railway}")
distance_highway = calculate_distance(launch_site[0], launch_site[1], highway_coord[0], highway_coord[1])
print(f"Highway: {distance_highway}")
distance_city = calculate_distance(launch_site[0], launch_site[1], city_coord[0], city_coord[1])
print(f"City: {distance_city}")

Railway: 1.2483887696356357
Highway: 0.6657895788690557
City: 23.259736967724265


In [19]:
# Draw lines between the launch site CCAFS SLC-40 and the different landmarks
distance_marker_railway = folium.Marker(
    railway_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color=#d35000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
distance_marker_highway = folium.Marker(
    highway_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color=#d35000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
distance_marker_city = folium.Marker(
    city_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color=#d35000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)

coordinates_railway = [launch_site,highway_coord]
coordinates_highwayway = [launch_site,railway_coord]
coordinates_city = [launch_site,city_coord]

lines_railway=folium.PolyLine(locations=coordinates_railway, weight=1)
lines_highway=folium.PolyLine(locations=coordinates_highwayway, weight=1)
lines_city=folium.PolyLine(locations=coordinates_city, weight=1)

site_map.add_child(lines_railway)
site_map.add_child(lines_highway)
site_map.add_child(lines_city)

In [20]:
distance_marker = folium.Marker(
    highway_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color=#d35000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
    - **Launch sites are in close proximity to railways. This will facilitate transportation of materials used in construction.**
*   Are launch sites in close proximity to highways?
    - **Similarly, they are close proximity to highways for transportation of personnel.**
*   Are launch sites in close proximity to coastline?
    - **There are launches that requires testing on drone ships as we saw from earlier data. However, the larger part of why launch sites are close to coastline would most likely be due to safety reasons, such as exploding rocket debris.**
*   Do launch sites keep certain distance away from cities?
    - **We see that launch sites are indeed far from cities. This is most likely due to keeping the disruptions to civilian lives to a minimum, as well as safety control.**

Also please try to explain your findings.
